In [5]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath('/home/cat/code/manifolds/')
sys.path.append(module_path)

from utils.wheel import wheel
from utils.calcium import calcium


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_69335/991363557.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
#############################################
######### LOAD SUITE2P BINARIZED DATA #######
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210214/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
#data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
data_dir = '/media/cat/4TB/donato/DON-006084/20210520/'

data_dir = '/home/cat/data/donato/DON-100795_22_08_05/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c.data_dir = data_dir
c.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/

#
c.detrend_model_order = 1

#
c.save_python = True

#
c.load_binarization()
traces = c.F_upphase_bin    # c.F_upphase_bin

print ("binarized data (# cells, # time points): ", traces.shape)


  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (155, 90000)
         self.Fneu (neuropile):  (155, 90000)
         self.iscell (Suite2p cell classifier output):  (246, 2)
              of which number of good cells:  (155,)
         self.spks (deconnvoved spikes):  (155, 90000)
         self.stat (footprints structure):  (155,)
         mean std over all cells :  33.5

  Binarization parameters: 
        low pass filter low cuttoff:  1 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_onphase:  1.5
        min_thresh_std_upphase:  1.5
        min_width_event_onphase:  15
        min_width_event_upphase:  7
        min_width_event_oasis:  2
        min_event_amplitude:  1


low pass filter: 100%|███████████████████████| 155/155 [00:00<00:00, 930.17it/s]
model filter: remove bleaching or trends: 100%|█| 155/155 [00:00<00:00, 1241.80i
binarizing continuous traces filtered fluorescence onphase:  54%|▌| 84/155 [00:0

In [35]:
#################################################
####### OPTIONAL: REMOVE DUPLICATE CELLS ########
#################################################
c.load_footprints()
c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
c.corr_max_percent_overlap = 0.25       # max normalized cell body allowed 
c.corr_threshold = 0.3                  # max correlation allowed for high overlap; 
                                        #     note correlations computed using filtered fluorescecne not binarized
c.corr_delete_method = 'lowest_snr'     # highest_connected: removes hub neurons,keeps more cells; 
                                        # lowest_snr - removes lower SNR cells, keep less neurons
c.recompute_deduplication = False        # recompute the dedplucaiton wif new paramters are saved
c.n_cores = 8                           #  Number of cores to use for multi processing

#
c.remove_duplicate_neurons()            

#       
traces_unique = traces[c.clean_cell_ids]
print ("All cells: ", c.F.shape, "  unique cells: ", traces_unique.shape)


 ... deduplicating cells... (can take ~5 for ~1,000cells) 


8001it [00:00, 97660.61it/s]                                                                                                                              

All cells:  (584, 55740)   unique cells:  (424, 55740)


In [36]:
###############################################################
### OPTIONAL: LOAD WHEEL DATA AND QUEISCENT OR RUN PERIODS ####
###############################################################
# 
w = wheel.Wheel()
w.root_dir = os.path.join(c.data_dir.replace('suite2p/','').replace('plane0',''),    # this assumes wheel data from Steffens' matlab scripts
                          'TRD-2P')                                                   #   is saved in the root diretory of the animal
w.load_track()
w.compute_velocity()
print ("Exp time : ", w.track.velocity.times.shape[0]/w.imaging_sample_rate)

# 
w.max_velocity_quiescent = 0.001  # in metres per second
idx_quiescent = w.get_indexes_quiescent_periods()

#
w.min_velocity_running = 0.1  # in metres per second
idx_run = w.get_indexes_run_periods()



Exp time :  1858.0
   Queiscent periods (seconds):  1359.2333333333333
   Running periods (seconds):  170.1


In [4]:
#########################################################
####### RUN PCA ON ALL OR SUBSET OF TRACES ##############
#########################################################
#

# take only 200 cells; and first 10 minutes
n_cells = 200
traces_unique = traces_unique[:n_cells]
print ("Selected cells (# cells, time points): ", traces_unique.shape)

# take quiescent periods
traces_in = traces_unique[:,idx_quiescent]

#
n_sec = 1300*30
times = np.arange(n_sec)
traces_in = traces_in[:,times]

#
suffix1 = 200   # # of cells
suffix2 = 600   # # seconds
recompute=True
pca, X_pca = c.compute_PCA(traces_in, suffix1,suffix2,recompute)

# plot  variance exaplined
var_exp = pca.explained_variance_ratio_
# 
fig=plt.figure()
ax=plt.subplot(111)
plt.plot(var_exp, label='variance explained')
plt.ylabel("variance explained")
plt.xlabel("PC component")
plt.ylim(0,0.05)
plt.legend()

ax2=ax.twinx()
cumsum = np.cumsum(var_exp)
cumsum = cumsum/np.max(cumsum)
plt.plot(cumsum, c='red', label='cumulative variance explained')
#plt.semilogy()
plt.legend()


plt.show()



NameError: name 'traces_unique' is not defined

In [63]:
# 
fig=plt.figure()
ax=plt.subplot(111)
plt.plot(var_exp, label='variance explained')
plt.ylabel("variance explained")
plt.xlabel("PC component")
#plt.ylim(0,0.05)
plt.legend()
plt.semilogy()

plt.show()
